# 14-03: systemd 与 launchd 服务管理Linux (systemd) 和 macOS (launchd) 的进程管理。

In [ ]:
// ========== 1. systemd 服务配置 ==========
// Linux 系统服务管理
/*
// /etc/systemd/system/myapp.service
[Unit]
Description=My Node.js Application
Documentation=https://example.com/docs
After=network.target
Wants=network.target

[Service]
Type=simple
User=myapp
Group=myapp
WorkingDirectory=/opt/myapp
ExecStart=/usr/bin/node /opt/myapp/dist/index.js
ExecReload=/bin/kill -HUP $MAINPID
Restart=on-failure
RestartSec=10
Environment=NODE_ENV=production
Environment=PORT=3000
# 或从文件加载环境变量
EnvironmentFile=/etc/myapp/env

# 资源限制
MemoryMax=512M
CPUQuota=50%
TasksMax=100

# 日志
StandardOutput=journal
StandardError=journal
SyslogIdentifier=myapp

[Install]
WantedBy=multi-user.target
*/
// 常用命令:
// sudo systemctl daemon-reload          # 重载配置
// sudo systemctl start myapp            # 启动服务
// sudo systemctl stop myapp             # 停止服务
// sudo systemctl restart myapp          # 重启服务
// sudo systemctl status myapp           # 查看状态
// sudo systemctl enable myapp           # 开机自启
// sudo systemctl disable myapp          # 禁用自启
// sudo journalctl -u myapp -f           # 查看日志

In [ ]:
// ========== 2. systemd Timer (定时任务) ==========
/*
// /etc/systemd/system/myapp-backup.timer
[Unit]
Description=Run backup daily

[Timer]
OnCalendar=daily
OnCalendar=Mon *-*-01 03:00:00  # 每月周一凌晨3点
Persistent=true

[Install]
WantedBy=timers.target
*/
/*
// /etc/systemd/system/myapp-backup.service
[Unit]
Description=MyApp Backup

[Service]
Type=oneshot
ExecStart=/opt/myapp/scripts/backup.sh
*/
// 命令:
// sudo systemctl start myapp-backup.timer
// sudo systemctl enable myapp-backup.timer
// systemctl list-timers

In [ ]:
// ========== 3. launchd (macOS) ==========
// macOS 使用 launchd 管理服务
/*
// ~/Library/LaunchAgents/com.example.myapp.plist (用户级)
// /Library/LaunchDaemons/com.example.myapp.plist (系统级)
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
    <key>Label</key>
    <string>com.example.myapp</string>
    
    <key>ProgramArguments</key>
    <array>
        <string>/usr/local/bin/node</string>
        <string>/Users/me/myapp/dist/index.js</string>
    </array>
    
    <key>WorkingDirectory</key>
    <string>/Users/me/myapp</string>
    
    <key>EnvironmentVariables</key>
    <dict>
        <key>NODE_ENV</key>
        <string>production</string>
        <key>PORT</key>
        <string>3000</string>
    </dict>
    
    <key>RunAtLoad</key>
    <true/>
    
    <key>KeepAlive</key>
    <true/>
    
    <key>StandardOutPath</key>
    <string>/Users/me/myapp/logs/out.log</string>
    
    <key>StandardErrorPath</key>
    <string>/Users/me/myapp/logs/error.log</string>
</dict>
</plist>
*/
// 常用命令:
// launchctl load ~/Library/LaunchAgents/com.example.myapp.plist
// launchctl unload ~/Library/LaunchAgents/com.example.myapp.plist
// launchctl start com.example.myapp
// launchctl stop com.example.myapp
// launchctl list | grep myapp
// tail -f ~/myapp/logs/out.log

In [ ]:
// ========== 4. launchd 定时任务 ==========
/*
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
    <key>Label</key>
    <string>com.example.myapp.backup</string>
    
    <key>ProgramArguments</key>
    <array>
        <string>/bin/sh</string>
        <string>/Users/me/myapp/scripts/backup.sh</string>
    </array>
    
    <!-- 每天凌晨3点执行 -->
    <key>StartCalendarInterval</key>
    <dict>
        <key>Hour</key>
        <integer>3</integer>
        <key>Minute</key>
        <integer>0</integer>
    </dict>
    
    <!-- 或间隔执行 (每3600秒) -->
    <!--
    <key>StartInterval</key>
    <integer>3600</integer>
    -->
</dict>
</plist>
*/

In [ ]:
// ========== 5. 使用 Node.js 管理服务 ==========
import { spawn } from 'child_process';
import { writeFileSync, mkdirSync } from 'fs';
import { homedir } from 'os';
import { join } from 'path';
class LaunchdService {
  private label: string;
  private plistPath: string;

  constructor(label: string) {
    this.label = label;
    this.plistPath = join(
      homedir(),
      'Library/LaunchAgents',
      `${label}.plist`
    );
  }

  create(config: {
    program: string;
    args: string[];
    workingDirectory: string;
    env?: Record<string, string>;
    keepAlive?: boolean;
    runAtLoad?: boolean;
  }): void {
    const plist = `<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
    <key>Label</key>
    <string>${this.label}</string>
    <key>ProgramArguments</key>
    <array>
        <string>${config.program}</string>
        ${config.args.map(arg => `<string>${arg}</string>`).join('\n        ')}
    </array>
    <key>WorkingDirectory</key>
    <string>${config.workingDirectory}</string>
    ${config.env ? `<key>EnvironmentVariables</key>
    <dict>
        ${Object.entries(config.env).map(([k, v]) => `<key>${k}</key><string>${v}</string>`).join('\n        ')}
    </dict>` : ''}
    <key>RunAtLoad</key>
    <${config.runAtLoad !== false}/>
    <key>KeepAlive</key>
    <${config.keepAlive !== false}/>
</dict>
</plist>`;

    mkdirSync(join(homedir(), 'Library/LaunchAgents'), { recursive: true });
    writeFileSync(this.plistPath, plist);
    console.log(`Created: ${this.plistPath}`);
  }

  load(): void {
    spawn('launchctl', ['load', this.plistPath], { stdio: 'inherit' });
  }

  unload(): void {
    spawn('launchctl', ['unload', this.plistPath], { stdio: 'inherit' });
  }

  start(): void {
    spawn('launchctl', ['start', this.label], { stdio: 'inherit' });
  }

  stop(): void {
    spawn('launchctl', ['stop', this.label], { stdio: 'inherit' });
  }
}
// 使用示例
// const service = new LaunchdService('com.example.myapp');
// service.create({
//   program: '/usr/local/bin/node',
//   args: ['dist/index.js'],
//   workingDirectory: '/Users/me/app',
//   env: { NODE_ENV: 'production' }
// });
// service.load();